In [1]:
from utils import *

In [82]:
member_with_daoid = pd.merge(members, daoinfo, on='daoid', how='outer')
proposals_with_members = pd.merge(proposals, member_with_daoid, left_on='space_id', right_on='governance', how='left')
proposals_first = proposals.groupby('space_id').first().reset_index()
proposals_first_with_members = pd.merge(proposals_first, member_with_daoid, left_on='space_id', right_on='governance', how='left')

### 1. Proposal Summary

In [131]:
total_votes_by_dao = votes.groupby(['dao_id']).size().reset_index()
total_votes_by_dao.columns = ['dao_id', 'total_votes']
total_votes_by_dao.to_csv('../data/total_votes_by_dao.csv', index=False)

In [132]:
total_unique_voters_by_dao = votes.groupby(['dao_id'])['voter_address'].nunique().reset_index()
total_unique_voters_by_dao.columns = ['dao_id', 'total_unique_voters']
total_unique_voters_by_dao.to_csv('../data/total_unique_voters_by_dao.csv', index=False)

In [133]:
total_proposals_by_dao = pd.read_csv('../data/total_proposals_by_dao.csv')

In [134]:
daoinfo_with_members = pd.read_csv('../data/daoinfo_with_members.csv')
daoinfo_with_members_proposals = pd.merge(daoinfo_with_members, total_votes_by_dao, on='dao_id', how='left')
daoinfo_with_members_proposals = pd.merge(daoinfo_with_members_proposals, total_unique_voters_by_dao, on='dao_id', how='left')
daoinfo_with_members_proposals = pd.merge(daoinfo_with_members_proposals, total_proposals_by_dao, on='dao_id', how='left')

daoinfo_with_members_proposals['votes_per_member'] = daoinfo_with_members_proposals['total_votes']/daoinfo_with_members_proposals['total_unique_voters']

daoinfo_with_members_proposals.to_csv('../data/daoinfo_with_members_proposals.csv', index=False)

Below are deprecated

In [13]:
proposals_first = proposals.groupby('space_id').first().reset_index()

In [31]:
proposal_summary = []

for index, row in proposals_first_with_members.iterrows():
    governance = row['space_id']
    total_proposal = row['total_proposal']
    dao_votes = votes.loc[votes['space_name']==governance]
    total_voters = dao_votes['voter'].nunique()
    total_votes = dao_votes.shape[0]
    if float(row['total_holders']) > 0:
        voting_pct = total_voters / float(row['total_holders'])
    else:
        voting_pct = None
    proposal_summary.append([governance, total_proposal, total_voters, total_votes, voting_pct])

In [140]:
colnames = ['governance', 'total_proposal', 'total_voters', 'total_votes', 'voting_pct']
df_member_data = pd.DataFrame(proposal_summary, columns=colnames)
df_member_data.to_csv('../data/proposal_summary.csv', index=False)

### 2. Proposal Details

In [35]:
votes.head()

,dao_id,proposal_id,vote_id,voter_address,voted_at,vote_choice,snapshot_space_id
0,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x3d887c23f894455ea744022fe55f3b638aae3ec208a3...,0x4A6737Da9668D09aCE702c3ff5e0dA33a84d28F7,1647993154,"{'3': 1, '4': 1, '5': 1, '6': 1, '7': 1}",banklessvault.eth
1,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x915a87e6f44b50abb06ef55277fd308146fd83b20d65...,0x540c44FF7B71F2487Cd11aaF53aeC1a85BFE8f7a,1647990901,"{'3': 5, '4': 40, '6': 5, '7': 50}",banklessvault.eth
2,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x96c8ce37e9bacc755d837540a4140a84917ddbbf15f1...,0xAf5c595eaA0ED462A75157513D99e1E3B0D1A570,1647986560,"{'3': 1, '5': 2, '7': 1}",banklessvault.eth
3,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x5ab41a959493b21ec0b69439584b36002d7af5d27c1c...,0x7d4D74853ED81b1f01bf0a5943Fe4b9459AFA036,1647985443,"{'3': 1, '4': 1, '5': 2}",banklessvault.eth
4,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x28cb75baac84568a1a85811e1404a254977ca9371ad0...,0x82c384FD188BFb22981B9922c67ae21A4493bF28,1647984979,{'6': 100},banklessvault.eth


In [34]:
proposals.head()

,dao_id,proposal_id,proposal_type,proposal_title,proposal_body,proposal_choices,proposal_start,proposal_end,proposal_block_level,proposal_state,proposal_author,snapshot_space_id,snapshot_space_name
0,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,weighted,Governance Solutions Engineer Re-implementatio...,**Title:** Governance Solutions Engineer Re-im...,"['mgoesdistance.eth', 'René', 'Saulthorin', '0...",1647388800,1647993600,14393704,closed,0xb6ac0341Fcf3FB507A8208D34a97f13779e1393D,banklessvault.eth,Bankless DAO
1,4,0x5f854c04a97238820aa729ab7a61797ba8f4da5ca9c5...,weighted,Grants Committe Election for S4 and S5,**Title:** Grants Committe Election for S4 and...,"['paul🏴.eth#1559', 'montgomery#8177', 'Freedom...",1647388800,1647993600,14393222,closed,0xb6ac0341Fcf3FB507A8208D34a97f13779e1393D,banklessvault.eth,Bankless DAO
2,4,0x98cde0a0de782f1963eb736f87ac406352154ae09cfb...,single-choice,Governance Solution Engineer Program - Re-Impl...,**Title:** Governance Solution Engineer Progra...,"['Approve', 'Deny']",1647388800,1647993600,14392917,closed,0xb6ac0341Fcf3FB507A8208D34a97f13779e1393D,banklessvault.eth,Bankless DAO
3,4,0x2b1f2678546586e7e12a75e71191292999a5568d4a95...,single-choice,BalancerDAO’s collaboration with BanklessDAO,**Title:** BalancerDAO’s collaboration with Ba...,"['Approve', 'Deny']",1646676000,1647277200,14340830,closed,0xb6ac0341Fcf3FB507A8208D34a97f13779e1393D,banklessvault.eth,Bankless DAO
4,4,0x1c2f7e4cce902906598be0b2c6e7b32762722d296a3f...,single-choice,Tokenomics - Migrate DAO-Owned Liquidity to U...,**Proposal Name:** Tokenomics - Migrate DAO-Ow...,"['Approve', 'Deny']",1646442000,1647046800,14321957,closed,0xb6ac0341Fcf3FB507A8208D34a97f13779e1393D,banklessvault.eth,Bankless DAO


In [33]:
proposals['proposal_type']

0            weighted
1            weighted
2       single-choice
3       single-choice
4       single-choice
            ...      
1363    single-choice
1364    single-choice
1365    single-choice
1366    single-choice
1367    single-choice
Name: proposal_type, Length: 1368, dtype: object

In [50]:
proposal_votes

,dao_id,proposal_id,vote_id,voter_address,voted_at,vote_choice,snapshot_space_id
0,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x3d887c23f894455ea744022fe55f3b638aae3ec208a3...,0x4A6737Da9668D09aCE702c3ff5e0dA33a84d28F7,1647993154,"{'3': 1, '4': 1, '5': 1, '6': 1, '7': 1}",banklessvault.eth
1,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x915a87e6f44b50abb06ef55277fd308146fd83b20d65...,0x540c44FF7B71F2487Cd11aaF53aeC1a85BFE8f7a,1647990901,"{'3': 5, '4': 40, '6': 5, '7': 50}",banklessvault.eth
2,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x96c8ce37e9bacc755d837540a4140a84917ddbbf15f1...,0xAf5c595eaA0ED462A75157513D99e1E3B0D1A570,1647986560,"{'3': 1, '5': 2, '7': 1}",banklessvault.eth
3,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x5ab41a959493b21ec0b69439584b36002d7af5d27c1c...,0x7d4D74853ED81b1f01bf0a5943Fe4b9459AFA036,1647985443,"{'3': 1, '4': 1, '5': 2}",banklessvault.eth
4,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x28cb75baac84568a1a85811e1404a254977ca9371ad0...,0x82c384FD188BFb22981B9922c67ae21A4493bF28,1647984979,{'6': 100},banklessvault.eth
...,...,...,...,...,...,...,...
264,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,QmdywqZdZoGMTwfQUnoWH76uiqVo5gW5fD5qwLx7JRRh11,0x148b8432C706c8579Ea5088EF7462527868D7497,1647392380,"{'2': 9000, '3': 9000, '4': 9000, '7': 9000}",banklessvault.eth
265,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x0f3b22ef794db844e589973ec5cbe6fa4ac5f458864b...,0xB04E6891e584F2884Ad2ee90b6545ba44F843c4A,1647392161,{'5': 1},banklessvault.eth
266,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x3b32980b0a7d1884a5c88fa1af3d859846ce5a1c43d0...,0x117e1EbB7D05545064850513021dF6ADe3C1690B,1647391065,{'3': 100},banklessvault.eth
267,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,0x01500416903dfe7f1f52846cf01ba9dac8b806bbc39d...,0xB72eD8401892466Ea8aF528C1af1d0524bc5e105,1647389818,{'7': 1},banklessvault.eth


In [126]:
proposal_details = []
for index, row in proposals.iterrows():
    dao_id = row['dao_id']
    try:
        token_address = daoinfo_with_members_proposals.loc[daoinfo_with_members_proposals['dao_id']==dao_id]['token_address'].values[0]
    except:
        token_address = math.nan
    block_height = row['proposal_block_level']
    proposal_id = row['proposal_id']
    proposal_type = row['proposal_type']
    proposal_block_level = row['proposal_block_level']
    proposal_votes = votes.loc[votes['proposal_id']==proposal_id]
    unique_voters = proposal_votes['voter_address'].nunique()
    total_votes = proposal_votes.shape[0]
    token_rate, voter_rate, agreement_coefficient = None, None, None
    try:
        if proposal_type != 'ranked-choice':
            proposal_res = proposal_votes.groupby('vote_choice')['voter_address'].apply(list).reset_index(name='voter_list')
            if proposal_res.shape[0] >= 1 and not isNaN(token_address):
                token_rate, voter_rate, agreement_coefficient = calc_poll_detail_stats(proposal_res, token_address, block_height)
    except:
        print('sth wrong', dao_id, proposal_id)
    data = [dao_id, proposal_id, unique_voters, total_votes, token_rate, voter_rate, agreement_coefficient]
    proposal_details.append(data)

request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succeeded!
request succ

In [127]:
colnames_proposal_detail = ['dao_id', 'proposal_id', 'unique_voters', 'total_votes', 'token_rate', 'voter_rate', 'agreement_coefficient']
df_proposal_details = pd.DataFrame(proposal_details, columns=colnames_proposal_detail)
df_proposal_details.to_csv('../data/proposal_details.csv', index=False)

In [130]:
snapshot_proposals_with_detail = pd.merge(proposals, df_proposal_details, on=['dao_id','proposal_id'], how='left')
snapshot_proposals_with_detail.to_csv('../data/snapshot_proposals_with_detail.csv', index=False)

In [138]:
df_proposal_details = pd.read_csv('../data/proposal_details.csv')

df = df_proposal_details[~df_proposal_details['token_rate'].isna()]
wm = lambda x: np.average(x, weights=df.loc[x.index, "total_votes"])


df_proposal_details_stats_by_dao = df.groupby(["dao_id"]).agg(adjusted_lots=("total_votes", "sum"),  
                          token_rate_mean=("token_rate", wm),
                          voter_rate_mean=("voter_rate", wm),
                          agreement_coefficient_mean=("agreement_coefficient", wm)).reset_index()

daoinfo_with_members_proposals_detail = pd.merge(daoinfo_with_members_proposals, df_proposal_details_stats_by_dao, on='dao_id', how='left')
daoinfo_with_members_proposals_detail.to_csv('../data/daoinfo_complete.csv', index=False)

In [139]:
daoinfo_with_members_proposals_detail.head().T.reset_index()

,index,0,1,2,3,4
0,title_x,Developer DAO,MetaversePRO,Constitution DAO,Bankless DAO,Friends With Benefits
1,treasury_1,NaN,NaN,0xb1c95ac257029d11f3f64ac67b2307a426699322,0xf26d1bb347a59f6c283c53156519cc1b1abaca51,NaN
2,treasury_1_type,NaN,NaN,GnosisSafeProxy,banklessvault,NaN
3,treasury_2,NaN,NaN,NaN,NaN,NaN
4,treasury_2_type,NaN,NaN,NaN,NaN,NaN
5,treasury_3,NaN,NaN,NaN,NaN,NaN
6,treasury_3_type,NaN,NaN,NaN,NaN,NaN
7,governance,NaN,NaN,2a988310-48d5-4809-bc65-622d43fc5f58,banklessvault.eth,friendswithbenefits.eth
8,governance_type,NaN,NaN,Assetsonly Governance,Snapshot,Snapshot
9,governance_url,NaN,NaN,NaN,https://snapshot.org/#/banklessvault.eth,https://snapshot.org/#/friendswithbenefits.eth


In [123]:
df_proposal_details[~df_proposal_details['token_rate'].isna()]

,dao_id,proposal_id,unique_voters,total_votes,token_rate,voter_rate,agreement_coefficient
0,4,0xb93ef13c7f48841d50191124a6e76c3bdd099edc501b...,269,269,0.058670,0.003717,15.782279
1,4,0x5f854c04a97238820aa729ab7a61797ba8f4da5ca9c5...,280,280,0.112663,0.021429,5.257597
8,4,0x33f869377c1b93a3b6db367001b6bb2604626f919ca4...,207,207,0.079404,0.028986,2.739439
10,4,0xeb3b1ff85d227856f9fab7f9af1b662a0013f19d58fb...,276,276,0.075025,0.003623,20.706805
36,5,0xeb5d43df11566511d48517371a1fd2110eec16cd0f13...,260,260,0.110856,0.019231,5.764497
56,5,QmZ9koHApY6wEXajD1KJsRca7Gp5jahArTbitHRHZ2qq1W,157,157,0.213322,0.184713,1.154883
330,11,QmR1mSZaQP8HEMU8nSMeBM5pVvkGoGG3TVu2H5aDuXyB8m,1632,1632,0.579254,0.710172,0.815654
468,16,0x05289873f68543c6fc518b0ae3598c82c51a3e8be98c...,53,53,0.404428,0.018868,21.434710
552,16,0xe3fff4b6a16f460fba6477b25fd531e7dd54cf898132...,42,42,0.339930,0.047619,7.138527
776,17,0xda01fd84639da8e06cacfa31f3e1aeb456daf06d86b7...,216,216,0.892684,0.819444,1.089377


In [105]:
def calc_poll_detail_stats(proposal_res, token_address, block_height):
    num_of_choices = proposal_res.shape[0]
    voter_lists = proposal_res['voter_list'].values

    r = r_token_holder_url(1, token_address, 100000, 0, block_height)
    poll_historical_balance = {}
    for i in r['data']['items']:
        poll_historical_balance[i['address']] = int(i['balance'])

    poll_stats = []
    prev_max_balance = 0
    win_choice_id = -1
    total_token = 0
    for j in range(len(voter_lists)):
        temp = {
            'voters': len(voter_lists[j]),
            'tokens': 0
        }
        for a in voter_lists[j]:
            if a.lower() in poll_historical_balance.keys():
                temp['tokens'] += poll_historical_balance[a.lower()]
                total_token += poll_historical_balance[a.lower()]
                if temp['tokens'] > prev_max_balance:
                    win_choice_id = j
                    prev_max_balance = temp['tokens']
        poll_stats.append(temp)
    
    winner = poll_stats[win_choice_id]
    
    
    if total_token > 0 and winner['voters'] > 0 and total_votes > 0:
        token_rate = winner['tokens']/total_token
        voter_rate = winner['voters']/total_votes
        agreement_coefficient = token_rate/voter_rate
    else:
        agreement_coefficient = math.nan
        token_rate = math.nan
        voter_rate = math.nan
    return token_rate, voter_rate, agreement_coefficient

In [80]:
poll_detail_stats, agreement_coefficient = calc_poll_detail_stats(proposal_res)

request succeeded!


In [83]:
agreement_coefficient

15.78227946681467

In [138]:
proposal_details = []
for index, row in proposals_with_members.tail(100).iterrows():
    governance = row['space_id']
    print(governance)
    proposal_id = row['id']
    block_height = row['snapshot']
    token_address = row['token_address']
    proposal_votes = votes.loc[votes['proposal']==proposal_id]
    print(proposal_votes.shape)
    unique_voters = proposal_votes['voter'].nunique()
    total_votes = proposal_votes.shape[0]
    choices = proposal_votes['choice'].values
    poll_detail_stats = None
    if len(choices) > 0:
        if len(proposal_votes['choice'].values[0]) == 1 and len(proposal_votes['choice'].values[-1]) == 1:
            proposal_res = proposal_votes.groupby('choice')['voter'].apply(list).reset_index(name='voter_list')
            poll_detail_stats, agreement_coefficient = calc_poll_detail_stats(proposal_res)
    data = [governance, proposal_id, unique_voters, total_votes, poll_detail_stats, agreement_coefficient]
    proposal_details.append(data)

jbdao.eth
(60, 7)
request succeeded!
jbdao.eth
(69, 7)
request succeeded!
jbdao.eth
(67, 7)
request succeeded!
jbdao.eth
(44, 7)
request succeeded!
jbdao.eth
(49, 7)
request succeeded!
jbdao.eth
(42, 7)
request succeeded!
jbdao.eth
(45, 7)
request succeeded!
jbdao.eth
(41, 7)
request succeeded!
jbdao.eth
(41, 7)
request succeeded!
jbdao.eth
(38, 7)
request succeeded!
jbdao.eth
(46, 7)
request succeeded!
jbdao.eth
(41, 7)
request succeeded!
jbdao.eth
(36, 7)
request succeeded!
jbdao.eth
(34, 7)
request succeeded!
jbdao.eth
(69, 7)
request succeeded!
jbdao.eth
(33, 7)
request succeeded!
jbdao.eth
(30, 7)
request succeeded!
jbdao.eth
(28, 7)
request succeeded!
jbdao.eth
(51, 7)
request succeeded!
jbdao.eth
(26, 7)
request succeeded!
jbdao.eth
(27, 7)
request succeeded!
jbdao.eth
(25, 7)
request succeeded!
jbdao.eth
(25, 7)
request succeeded!
jbdao.eth
(26, 7)
request succeeded!
jbdao.eth
(21, 7)
request succeeded!
jbdao.eth
(23, 7)
request succeeded!
jbdao.eth
(30, 7)
request succeeded!
j

In [24]:
member_with_daoid = pd.merge(members, df[['title','governance']], on='title', how='left')
proposals_with_members = pd.merge(proposals, member_with_daoid, left_on='space_id', right_on='governance', how='left')